**By Peter A. Stokes, École Pratique des Hautes Études – Université PSL**

_These are brief notes and exercises on working with images using Python. They are intended as complements to classroom teaching. They assume a a basic level of Python programming. This notebook also assumes that the relevant Numpy and SciKit libraries have already been installed in your Python system._

_If you are viewing this in Jupyter then you can edit the code simply by typing in the boxes. You can also execute the code in any box by clicking on the box and typing SHIFT + ENTER or using the 'Run' button in the menubar above._

# The Scikit-Image Library

The Scikit-image library is a standard library for data science and manipulation relating to images, which allows us to work directly with the statistics of the image itself. The purpose of this notebook is to give you a basic introduction to the library and how to use it.

In practice, the Python Imaging Library (PIL) is sometimes easier to use for basic (and even relatively advanced) image manipulation. In practice, it is best not to mix PIL and scikit-image if possible, so you should try to evaluate from the start which library best suits your needs. (In fact, scikit-image often simply calls PIL more or less directly, but it is still cleaner to use one or the other in your code.) For a parallel notebook using PIL, see https://github.com/pastokes/MS-images

Note that Scikit-Image is often abbreviated as skimage, and we will use this shorter form throughout this notebook.

## Importing libraries

The first step in any realistic Python programme is to import the necessary libraries. Here, we will mainly be using parts of scikit-image, as well as matplotlib so that we can see the results of our work. For the purposes of this exercise, the imports are added at the relevant part of the document, but normally it's good practice to have them all together at the start.

## Opening and saving image files; showing images

To load and save images we need to tell Python where the image can be found, and then load them using [scikit-image.io](https://scikit-image.org/docs/dev/api/skimage.io.html). We can use this to open a file which is already saved on our disk, or open one directly from the internet: `skimage.io.imread` takes care of all the work for us. We can also show the image using `skimage.io.imshow`.

This is how it works:

In [ ]:
from skimage import io

# This is the address of an image on Gallica
im_addr = "https://gallica.bnf.fr/iiif/ark:/12148/btv1b10024153f/f35/full/full/0/native.jpg"

# Now we read the image
im = io.imread(im_addr)
                
# Now we show the image
io.imshow(im)

Saving an image to our disk is equally easy in principle using `skimage.io.imsave`. However, there are some important points that you must keep in mind, particularly when working with Jupyter notebooks.

First, **be careful when saving images as you can easily overwrite your data by mistake**. If you try to save an image and a file exists already at that path and with that file name then **Python will simply overwrite your existing file without even asking**.

Second, keep in mind that Jupyter Notebooks normally save the file in the same directory as your notebook. This may not be obvious at first, particularly if you have your notebook somewhere unusual, but you should be aware of this. Of course you can always provide a full path to `imsave` if you want to be clear.

In [ ]:
# Let's save a copy of the image on your computer
io.imsave('newimage.jpg', im)

## The image as array

Internally, `io.imread` returns the image in the form of a Numpy `ndarray`. This is almost like a normal Python array, but it is much more efficient for calculations across the whole image (or array). We can use normal Python slices to extract parts of the image, for instance. To see the dimensions of the image, we use `ndarray.shape` which returns a tuple of aach of the different dimensions (height, width, colour depth). We can also use normal Python array slices to select a specific region of the image.

In [ ]:
# Print the shape of the image
print('Image dimensions:', im.shape)

# Note that the image has three colour channels (the last number in the tuple)
# Let's convert it to a grayscale image and see the difference
from skimage.color import rgb2gray

im_gray = rgb2gray(im)
print('Dimensions of grayscale image:', im_gray.shape)

# Let's show a region (slice) of the image
im_region = im[5950:7100, 2300:3500, :]
io.imshow(im_region)

In [ ]:
# Now let's take the three colour channels and look at them one by one
print('Red channel')
io.imshow(im_region[:,:,0])
print('Blue channel')
io.imshow(im_region[:,:,1])
print('Green channel')
io.imshow(im_region[:,:,2])

What happened? Was this what you expect? Probably not... How many images did you expect to see, and how many are there?

We asked to show three different plots, but in fact we only got one. This is because `imshow` uses the `matplotlib` library, which is very powerful but sometimes not so intuitive. In fact, we can use `matplotlib` directly to create complex figures before displaying them. Try this code, for example, to see what happens:

In [ ]:
from matplotlib import pyplot as plt

fig, axes = plt.subplots(1, 4, figsize=(16, 8))
ax = axes.ravel()

ax[0].imshow(im_region)
ax[0].set_title("Original image")
ax[1].imshow(im_region[:,:,0], cmap=plt.cm.gray)
ax[1].set_title("Red channel")
ax[2].imshow(im_region[:,:,1], cmap=plt.cm.gray)
ax[2].set_title("Green channel")
ax[3].imshow(im_region[:,:,2], cmap=plt.cm.gray)
ax[3].set_title("Blue channel")

fig.tight_layout()
plt.show()

The code for the previous block using `matplotlib` is a good example of reuse from the internet. As mentioned, `matplotlib` is not so intuitive at first, but we can find examples of code elsewhere which are close to what we want, and we can then adapt them for our needs. If you look at the [skimage tutorial on RGB to grayscale](https://scikit-image.org/docs/dev/auto_examples/color_exposure/plot_rgb_to_gray.html), you will see something almost identical to what we have here. Look now at the tutorial and the code here, and note the differences. Now, try changing our code in different ways to understand a bit more about how it works and how you might use it. Here is the code again, so you can change this version while keeping the original one for your reference:

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(16, 8))
ax = axes.ravel()

ax[0].imshow(im_region)
ax[0].set_title("Original image")
ax[1].imshow(im_region[:,:,0], cmap=plt.cm.gray)
ax[1].set_title("Red channel")
ax[2].imshow(im_region[:,:,1], cmap=plt.cm.gray)
ax[2].set_title("Green channel")
ax[3].imshow(im_region[:,:,2], cmap=plt.cm.gray)
ax[3].set_title("Blue channel")

fig.tight_layout()
plt.show()

## The Image Histogram

As well as the dimensions, another important part of the statistics of an image is the histogram. There are several different ways of finding the histogram of an array or ndarray in Python, and one of these is `histogram` in the `skimage.exposure` library.

The way this works is that we pass it the image, and it returns two different values (in a tuple). The first is the histogram itself, which is an array of the histogram values, and the second is the centre-points of the histogram bins (in practice, this is basically the labels for the x-axis of the histogram).

Let's try it.

In [ ]:
from skimage.exposure import histogram

hist, hist_centers = histogram(im_region)
plt.plot(hist_centers, hist)

Notice the warning here. Do you understand it? Let's do it again, this time taking each channel at a time.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 8))
ax = axes.ravel()

hist, hist_centers = histogram(im_region[:,:,0])
ax[0].plot(hist_centers, hist)
ax[0].set_title("Red histogram")
hist, hist_centers = histogram(im_region[:,:,1])
ax[1].plot(hist_centers, hist)
ax[1].set_title("Green histogram")
hist, hist_centers = histogram(im_region[:,:,2])
ax[2].plot(hist_centers, hist)
ax[2].set_title("Blue histogram")

fig.tight_layout()
plt.show()

This code works but is a bit inefficient as it is repetitive, and if we want to change anything then we have to do it three times. As an exercise, try rewriting it to use a function and `for` loop instead, and then update the rest of this Jupyter Notebook to use the new function.

## Basic Image Manipulation

Now let's try some basic image manipulation. First, we can try equalising the histogram to enhance the differences between similar colours:

In [ ]:
from skimage.exposure import equalize_hist, equalize_adapthist, rescale_intensity

fig, axes = plt.subplots(1, 4, figsize=(16, 8))
ax = axes.ravel()

ax[0].imshow(im_region)
ax[0].set_title("Original image")
ax[1].imshow(rescale_intensity(im_region))
ax[1].set_title("Intensity rescaled")
ax[2].imshow(equalize_adapthist(im_region))
ax[2].set_title("Histograms equalised adaptively")
ax[3].imshow(equalize_hist(im_region))
ax[3].set_title("Histograms equalised")

fig.tight_layout()
plt.show()


A number of other methods are available: have a look at the [skimage.exposure](https://scikit-image.org/docs/dev/api/skimage.exposure.html) documentation and the [Contrast and Exposure](https://scikit-image.org/docs/dev/user_guide/transforming_image_data.html?highlight=enhance#contrast-and-exposure) section of the skimage user guide, and try out some of their techniques (for instance `adjust_gamma` or `adjust_log`).

What about the histograms? Write the code here to display the histograms of the different images after processing. Do you understand what has happened? The one for histogram equalisation is done for you; you should add the others yourself.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
ax = axes.ravel()

im_eq = equalize_hist(im_region)

hist, hist_centers = histogram(im_eq[:,:,0])
ax[0].plot(hist_centers, hist)
ax[0].set_title("Red equalised histogram")
hist, hist_centers = histogram(im_eq[:,:,1])
ax[1].plot(hist_centers, hist)
ax[1].set_title("Green equalised histogram")
hist, hist_centers = histogram(im_eq[:,:,2])
ax[2].plot(hist_centers, hist)
ax[2].set_title("Blue equalised histogram")

fig.tight_layout()
plt.show()

Now experiment yourself to be sure you understand what's going on:

## Image Feature Extraction

As we discussed in previous classes, people have created a large number of 'hand-crafted' features that can then be used (for instance) in training neural networks. This is becoming less common, since it turns out that it is usually more effective to let the network develop its own features. Nevertheless it can still be useful, and the [skimage.feature library](https://scikit-image.org/docs/dev/api/skimage.feature.html) gives us a good starting-point. Let's try some of them now.

First, let's try the Canny edge detector, to find the edges of our image. Note that `feature.canny` requires a *grayscale* image (as specified in [the documentation](https://scikit-image.org/docs/dev/api/skimage.feature.html?highlight=canny#skimage.feature.canny)), so we have to convert it first. (What happens if we don't?)

In [ ]:
from skimage import feature

im_gray = rgb2gray(im_region)

im_canny = feature.canny(im_gray)
io.imshow(im_canny)

This isn't very meaningful! Let's try with a different image:

In [ ]:
# Here is a random image I found on the internet...
im_addr = "https://happywall-img-gallery.imgix.net/2132/geometric_forest_display.jpg"

im = io.imread(im_addr)
im_canny = feature.canny(rgb2gray(im))

fig, axes = plt.subplots(1, 2, figsize=(16, 8))
ax = axes.ravel()

ax[0].imshow(im)
ax[1].imshow(im_canny, cmap=plt.cm.gray_r)

ax[0].set_title("Original image")
ax[1].set_title("Canny edges")

fig.tight_layout()
plt.show()

We can even do more fancy things like find the corners on the image using a corner detector. (This code is adapted from the [Corner detection](https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_corner.html) page in the skimage documentation).

In [ ]:
# Import the relevant libraries
from skimage.feature import corner_harris, corner_peaks

# The corner detector requires a 2d image, so let's convert to grayscale
im_gray = rgb2gray(im)

# Now let's get the Harris corner response
harris = corner_harris(im_gray)

# Now we want to find the peaks of the corner response
# The result should be the coordinates of all the corners in the original image
coords = corner_peaks(harris, min_distance=5, threshold_rel=0.002)

# Now show the results. Here we add two layers to the same figure:
# one with the image, and one with the crosses which should be at the corners.
fig, ax = plt.subplots()
ax.imshow(im)  # Adds the image to the figure
ax.plot(coords[:, 1], coords[:, 0], '+r', markersize=15)

plt.show()

Did it work? Try changing the parameters of the corner peak detector, and try also using different corner detectors (such as Foerstner or FAST) to see which one works the best.

# Going Further

This gives you some idea of the possibilities that Python allows. Some things that you can now do include:

* Find all the images in a TEI XML file, make copies of the images, and convert the copies into black and white.
* Find the addresses of all images tagged with a given attribute in a TEI file and automatically adjust the image contrast and histogram of those images, saving the results in a new directory.
* Going through a directory and automatically convert all JPG files in the directory to thumbnails. (For instructions on how to process all the files in a directory, see ['Automatic Batch Processing of a Set of Images'](https://github.com/pastokes/MS-images/blob/master/1.%20Image%20Analysis%20with%20PIL.ipynb) in the worksheet that I prepared for a different course.

And so on. We will see more possibilities in the following worksheets, but in the meantime, use your imagination and don't be afraid to play and see how things work.


---
![Licence Creative Commons](https://i.creativecommons.org/l/by/4.0/88x31.png)
This work (the contents of this Jupyter Python notebook) is licenced under a [Creative Commons Attribution 4.0 International](http://creativecommons.org/licenses/by/4.0/)